In [ ]:
#!pip install langchain langchain_community
#!pip install neo4j
#!pip install groq
#!pip install -U --quiet langchain-groq
#!pip install langchain_experimental
!pip install together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 3.0 MB/s eta 0:00:00


In [ ]:
NEO4J_PASSWORD = ""
OPENAI_API_KEY = ""
NEO4J_URL = ""
NEO4J_USERNAME = ""

In [ ]:
## Questions
import pandas as pd
questions_and_queries = [
    # Questions about the Virus
    {"question": "What are all the viruses mentioned in the dataset?", "query": "MATCH (v:Virus) RETURN v.name"},
    {"question": "Which symptoms are caused by the virus Mpox?", "query": "MATCH (v:Virus {name: 'Mpox'})-[:CAUSES]->(s:Symptom) RETURN s.name"},
    {"question": "What locations are affected by Mpox?", "query": "MATCH (v:Virus {name: 'Mpox'})-[:AFFECTS]->(l:Location) RETURN l.name, l.country"},
    {"question": "What transmission methods are related to Mpox?", "query": "MATCH (v:Virus {name: 'Mpox'})-[:SPREADS_BY]->(t:Transmission) RETURN t.method"},
    {"question": "Which organizations are involved in Mpox?", "query": "MATCH (v:Virus {name: 'Mpox'})<-[:INVOLVED_IN]-(o:Organization) RETURN o.name"},

    # Questions about Symptoms
    {"question": "What are all the symptoms in the dataset?", "query": "MATCH (s:Symptom) RETURN s.name"},
    {"question": "Which viruses cause the symptom 'Fever'?", "query": "MATCH (s:Symptom {name: 'Fever'})<-[:CAUSES]-(v:Virus) RETURN v.name"},
    {"question": "Which articles mention the symptom 'Rash'?", "query": "MATCH (a:Article)-[:MENTIONS_SYMPTOM]->(s:Symptom {name: 'Rash'}) RETURN a.url"},
    {"question": "How many viruses cause each symptom?", "query": "MATCH (v:Virus)-[:CAUSES]->(s:Symptom) RETURN s.name, count(v) AS virus_count"},

    # Questions about Locations
    {"question": "What are all the locations in the dataset?", "query": "MATCH (l:Location) RETURN l.name, l.country"},
    {"question": "Which viruses affect Mexico?", "query": "MATCH (l:Location {name: 'Mexico'})<-[:AFFECTS]-(v:Virus) RETURN v.name"},
    {"question": "Which articles mention Mexico?", "query": "MATCH (a:Article)-[:MENTIONS_LOCATION]->(l:Location {name: 'Mexico'}) RETURN a.url"},
    {"question": "What locations are affected by more than one virus?", "query": "MATCH (v:Virus)-[:AFFECTS]->(l:Location) WITH l, COUNT(v) AS virus_count WHERE virus_count > 1 RETURN l.name, virus_count"},

    # Questions about Transmission Methods
    {"question": "What are all the transmission methods in the dataset?", "query": "MATCH (t:Transmission) RETURN t.method"},
    {"question": "Which viruses are spread by 'Airborne Transmission'?", "query": "MATCH (v:Virus)-[:SPREADS_BY]->(t:Transmission {method: 'Airborne Transmission'}) RETURN v.name"},
    {"question": "Which articles mention 'Direct Contact' as a transmission method?", "query": "MATCH (a:Article)-[:MENTIONS_TRANSMISSION]->(t:Transmission {method: 'Direct Contact'}) RETURN a.url"},
    {"question": "How many viruses are transmitted by each method?", "query": "MATCH (v:Virus)-[:SPREADS_BY]->(t:Transmission) RETURN t.method, count(v) AS virus_count"},

    # Questions about Organizations
    {"question": "What are all the organizations involved in viruses?", "query": "MATCH (o:Organization) RETURN o.name"},
    {"question": "Which organizations are involved in the virus Mpox?", "query": "MATCH (o:Organization)-[:INVOLVED_IN]->(v:Virus {name: 'Mpox'}) RETURN o.name"},
    {"question": "Which articles are published by the organization 'WHO'?", "query": "MATCH (a:Article)-[:PUBLISHED_BY]->(o:Organization {name: 'WHO'}) RETURN a.url"},

    # Questions about Articles
    {"question": "What are all the articles in the dataset?", "query": "MATCH (a:Article) RETURN a.url, a.abstract, a.label"},
    {"question": "Which articles are labeled as true?", "query": "MATCH (a:Article {label: 'True'}) RETURN a.url, a.abstract"},
    {"question": "Which articles are labeled as fake?", "query": "MATCH (a:Article {label: 'Fake'}) RETURN a.url, a.abstract"},
    {"question": "Which virus is mentioned in the article with the URL 'example.com'?", "query": "MATCH (a:Article {url: 'example.com'})-[:ABOUT_VIRUS]->(v:Virus) RETURN v.name"},
    {"question": "Which symptoms are mentioned in the article with the URL 'example.com'?", "query": "MATCH (a:Article {url: 'example.com'})-[:MENTIONS_SYMPTOM]->(s:Symptom) RETURN s.name"},

    # More specific combinations of queries
    {"question": "What are the symptoms caused by Mpox in Mexico?", "query": "MATCH (v:Virus {name: 'Mpox'})-[:CAUSES]->(s:Symptom)<-[:MENTIONS_SYMPTOM]-(a:Article)-[:MENTIONS_LOCATION]->(l:Location {name: 'Mexico'}) RETURN s.name"},
    {"question": "Which transmission methods are mentioned in articles about Mpox in Mexico?", "query": "MATCH (a:Article)-[:ABOUT_VIRUS]->(v:Virus {name: 'Mpox'})-[:SPREADS_BY]->(t:Transmission) WHERE (a)-[:MENTIONS_LOCATION]->(l:Location {name: 'Mexico'}) RETURN t.method"},
    {"question": "Which organizations are involved in publishing articles about Mpox?", "query": "MATCH (a:Article)-[:ABOUT_VIRUS]->(v:Virus {name: 'Mpox'})-[:PUBLISHED_BY]->(o:Organization) RETURN o.name, a.url"},

    # More exploratory questions
    {"question": "How many articles mention Mpox?", "query": "MATCH (a:Article)-[:ABOUT_VIRUS]->(v:Virus {name: 'Mpox'}) RETURN count(a)"},
    {"question": "How many symptoms are associated with each virus?", "query": "MATCH (v:Virus)-[:CAUSES]->(s:Symptom) RETURN v.name, count(s) AS symptom_count"},
    {"question": "Which virus spreads through 'Animal Transmission'?", "query": "MATCH (v:Virus)-[:SPREADS_BY]->(t:Transmission {method: 'Animal Transmission'}) RETURN v.name"},

    # Example of abstract-based queries
    {"question": "What is the abstract of the article with the URL 'example.com'?", "query": "MATCH (a:Article {url: 'example.com'}) RETURN a.abstract"},
    {"question": "How many articles are labeled as fact?", "query": "MATCH (a:Article {label: 'True'}) RETURN count(a)"},

    # And more such combinations...
]
questions_and_queries += [
    # Virus-related questions (continued)
    {"question": "Which viruses are mentioned in the dataset?", "query": "MATCH (v:Virus) RETURN v.name"},
    {"question": "Which viruses affect multiple locations?", "query": "MATCH (v:Virus)-[:AFFECTS]->(l:Location) WITH v, COUNT(l) AS location_count WHERE location_count > 1 RETURN v.name, location_count"},
    {"question": "Which viruses are spread by multiple transmission methods?", "query": "MATCH (v:Virus)-[:SPREADS_BY]->(t:Transmission) WITH v, COUNT(t) AS method_count WHERE method_count > 1 RETURN v.name, method_count"},
    {"question": "Which viruses are involved in more than one article?", "query": "MATCH (v:Virus)<-[:ABOUT_VIRUS]-(a:Article) WITH v, COUNT(a) AS article_count WHERE article_count > 1 RETURN v.name, article_count"},
    {"question": "What is the total number of viruses in the dataset?", "query": "MATCH (v:Virus) RETURN COUNT(v) AS virus_count"},

    # Symptom-related questions (continued)
    {"question": "Which symptom is most frequently mentioned in articles?", "query": "MATCH (a:Article)-[:MENTIONS_SYMPTOM]->(s:Symptom) RETURN s.name, COUNT(a) AS mention_count ORDER BY mention_count DESC LIMIT 1"},
    {"question": "Which viruses cause both 'Fever' and 'Rash'?", "query": "MATCH (v:Virus)-[:CAUSES]->(s:Symptom) WHERE s.name IN ['Fever', 'Rash'] RETURN v.name"},
    {"question": "Which symptoms are related to viruses that affect Mexico?", "query": "MATCH (v:Virus)-[:CAUSES]->(s:Symptom) WHERE (v)-[:AFFECTS]->(l:Location {name: 'Mexico'}) RETURN DISTINCT s.name"},
    {"question": "How many symptoms are mentioned in each article?", "query": "MATCH (a:Article)-[:MENTIONS_SYMPTOM]->(s:Symptom) RETURN a.url, COUNT(s) AS symptom_count"},
    {"question": "Which viruses cause symptoms related to respiratory issues?", "query": "MATCH (s:Symptom) WHERE s.name CONTAINS 'Respiratory' WITH s MATCH (v:Virus)-[:CAUSES]->(s) RETURN v.name"},

    # Location-related questions (continued)
    {"question": "Which locations are mentioned in more than one article?", "query": "MATCH (a:Article)-[:MENTIONS_LOCATION]->(l:Location) WITH l, COUNT(a) AS article_count WHERE article_count > 1 RETURN l.name, article_count"},
    {"question": "Which viruses affect locations in both North America and Europe?", "query": "MATCH (v:Virus)-[:AFFECTS]->(l:Location) WHERE l.country IN ['United States', 'Mexico', 'Canada', 'United Kingdom', 'France', 'Germany'] RETURN v.name, COUNT(l) AS location_count"},
    {"question": "Which countries have the most virus-related articles?", "query": "MATCH (a:Article)-[:MENTIONS_LOCATION]->(l:Location) RETURN l.country, COUNT(a) AS article_count ORDER BY article_count DESC"},
    {"question": "What are the top 3 locations most frequently affected by viruses?", "query": "MATCH (v:Virus)-[:AFFECTS]->(l:Location) RETURN l.name, COUNT(v) AS virus_count ORDER BY virus_count DESC LIMIT 3"},
    {"question": "Which articles mention locations in Asia?", "query": "MATCH (a:Article)-[:MENTIONS_LOCATION]->(l:Location) WHERE l.country IN ['China', 'India', 'Japan'] RETURN a.url"},

    # Transmission-related questions (continued)
    {"question": "Which transmission method is most common for viruses?", "query": "MATCH (v:Virus)-[:SPREADS_BY]->(t:Transmission) RETURN t.method, COUNT(v) AS virus_count ORDER BY virus_count DESC LIMIT 1"},
    {"question": "Which viruses are spread by both airborne and droplet transmission?", "query": "MATCH (v:Virus)-[:SPREADS_BY]->(t:Transmission) WHERE t.method IN ['Airborne', 'Droplet'] RETURN v.name"},
    {"question": "Which articles mention viruses that are spread by direct contact?", "query": "MATCH (a:Article)-[:ABOUT_VIRUS]->(v:Virus)-[:SPREADS_BY]->(t:Transmission {method: 'Direct Contact'}) RETURN a.url"},
    {"question": "Which transmission methods are mentioned in the most articles?", "query": "MATCH (a:Article)-[:MENTIONS_TRANSMISSION]->(t:Transmission) RETURN t.method, COUNT(a) AS mention_count ORDER BY mention_count DESC"},
    {"question": "How many different transmission methods are associated with each virus?", "query": "MATCH (v:Virus)-[:SPREADS_BY]->(t:Transmission) RETURN v.name, COUNT(t) AS method_count"},

    # Organization-related questions (continued)
    {"question": "Which organizations are involved in more than one virus?", "query": "MATCH (o:Organization)-[:INVOLVED_IN]->(v:Virus) WITH o, COUNT(v) AS virus_count WHERE virus_count > 1 RETURN o.name, virus_count"},
    {"question": "Which articles are published by more than one organization?", "query": "MATCH (a:Article)-[:PUBLISHED_BY]->(o:Organization) WITH a, COUNT(o) AS org_count WHERE org_count > 1 RETURN a.url, org_count"},
    {"question": "Which organizations are involved in articles about viruses spread by airborne transmission?", "query": "MATCH (a:Article)-[:ABOUT_VIRUS]->(v:Virus)-[:SPREADS_BY]->(t:Transmission {method: 'Airborne'})-[:PUBLISHED_BY]->(o:Organization) RETURN o.name"},
    {"question": "What is the total number of organizations involved in the dataset?", "query": "MATCH (o:Organization) RETURN COUNT(o) AS org_count"},
    {"question": "Which organizations are involved in viruses affecting both North and South America?", "query": "MATCH (o:Organization)-[:INVOLVED_IN]->(v:Virus)-[:AFFECTS]->(l:Location) WHERE l.country IN ['United States', 'Brazil'] RETURN DISTINCT o.name"},

    # Article-related questions (continued)
    {"question": "Which articles are about viruses that spread by both airborne and direct contact?", "query": "MATCH (a:Article)-[:ABOUT_VIRUS]->(v:Virus)-[:SPREADS_BY]->(t:Transmission) WHERE t.method IN ['Airborne', 'Direct Contact'] RETURN a.url"},
    {"question": "Which articles are related to more than one virus?", "query": "MATCH (a:Article)-[:ABOUT_VIRUS]->(v:Virus) WITH a, COUNT(v) AS virus_count WHERE virus_count > 1 RETURN a.url, virus_count"},
    {"question": "Which articles mention both 'Fever' and 'Rash' as symptoms?", "query": "MATCH (a:Article)-[:MENTIONS_SYMPTOM]->(s:Symptom) WHERE s.name IN ['Fever', 'Rash'] RETURN a.url"},
    {"question": "Which articles are published by more than one organization?", "query": "MATCH (a:Article)-[:PUBLISHED_BY]->(o:Organization) WITH a, COUNT(o) AS org_count WHERE org_count > 1 RETURN a.url, org_count"},
    {"question": "Which articles mention both Mexico and the United States?", "query": "MATCH (a:Article)-[:MENTIONS_LOCATION]->(l:Location) WHERE l.country IN ['Mexico', 'United States'] RETURN DISTINCT a.url"},

    # Combining questions (continued)
    {"question": "Which organizations publish articles about Mpox in Mexico?", "query": "MATCH (a:Article)-[:ABOUT_VIRUS]->(v:Virus {name: 'Mpox'})-[:PUBLISHED_BY]->(o:Organization) WHERE (a)-[:MENTIONS_LOCATION]->(l:Location {name: 'Mexico'}) RETURN o.name, a.url"},
    {"question": "Which viruses cause symptoms related to respiratory and cardiovascular issues?", "query": "MATCH (s:Symptom) WHERE s.name CONTAINS 'Respiratory' OR s.name CONTAINS 'Cardiovascular' WITH s MATCH (v:Virus)-[:CAUSES]->(s) RETURN v.name"},
    {"question": "Which transmission methods are mentioned in articles about viruses affecting both Mexico and the United States?", "query": "MATCH (a:Article)-[:ABOUT_VIRUS]->(v:Virus)-[:MENTIONS_TRANSMISSION]->(t:Transmission) WHERE (a)-[:MENTIONS_LOCATION]->(l:Location) AND l.country IN ['Mexico', 'United States'] RETURN t.method, COUNT(a)"},
    {"question": "Which symptoms are caused by viruses that are spread by airborne and droplet transmission?", "query": "MATCH (v:Virus)-[:CAUSES]->(s:Symptom) WHERE (v)-[:SPREADS_BY]->(t:Transmission) AND t.method IN ['Airborne', 'Droplet'] RETURN s.name"},
]

questions_and_queries += [
    # Continuing from the last question
    {"question": "Which organizations publish articles about viruses affecting both Mexico and Brazil?", "query": "MATCH (a:Article)-[:ABOUT_VIRUS]->(v:Virus)-[:PUBLISHED_BY]->(o:Organization) WHERE (a)-[:MENTIONS_LOCATION]->(l:Location) AND l.country IN ['Mexico', 'Brazil'] RETURN DISTINCT o.name, a.url"},
    {"question": "Which viruses cause the most symptoms?", "query": "MATCH (v:Virus)-[:CAUSES]->(s:Symptom) RETURN v.name, COUNT(s) AS symptom_count ORDER BY symptom_count DESC LIMIT 1"},
    {"question": "Which viruses affect the most countries?", "query": "MATCH (v:Virus)-[:AFFECTS]->(l:Location) RETURN v.name, COUNT(DISTINCT l.country) AS country_count ORDER BY country_count DESC LIMIT 1"},
    {"question": "Which organizations are involved in the most viruses?", "query": "MATCH (o:Organization)-[:INVOLVED_IN]->(v:Virus) RETURN o.name, COUNT(v) AS virus_count ORDER BY virus_count DESC LIMIT 1"},
    {"question": "Which articles mention viruses affecting more than one continent?", "query": "MATCH (a:Article)-[:MENTIONS_LOCATION]->(l:Location) WITH a, COUNT(DISTINCT l.country) AS location_count WHERE location_count > 1 RETURN a.url"},

    # Virus-specific combinations (continuing)
    {"question": "Which viruses are transmitted via multiple methods in more than one location?", "query": "MATCH (v:Virus)-[:SPREADS_BY]->(t:Transmission), (v)-[:AFFECTS]->(l:Location) WITH v, COUNT(DISTINCT t) AS method_count, COUNT(DISTINCT l) AS location_count WHERE method_count > 1 AND location_count > 1 RETURN v.name"},
    {"question": "Which symptoms are associated with viruses transmitted by direct contact?", "query": "MATCH (v:Virus)-[:SPREADS_BY]->(t:Transmission {method: 'Direct Contact'})-[:CAUSES]->(s:Symptom) RETURN DISTINCT s.name"},
    {"question": "What articles mention symptoms related to 'Cardiovascular' issues?", "query": "MATCH (a:Article)-[:MENTIONS_SYMPTOM]->(s:Symptom) WHERE s.name CONTAINS 'Cardiovascular' RETURN a.url"},
    {"question": "What viruses are transmitted via both 'Direct Contact' and 'Airborne' methods?", "query": "MATCH (v:Virus)-[:SPREADS_BY]->(t:Transmission) WHERE t.method IN ['Direct Contact', 'Airborne'] RETURN DISTINCT v.name"},
    {"question": "Which organizations publish articles about viruses affecting Europe?", "query": "MATCH (a:Article)-[:MENTIONS_LOCATION]->(l:Location) WHERE l.country IN ['United Kingdom', 'France', 'Germany']-[:PUBLISHED_BY]->(o:Organization) RETURN DISTINCT o.name, a.url"},

    # More combinations of virus and symptom questions
    {"question": "Which viruses cause both respiratory and gastrointestinal symptoms?", "query": "MATCH (v:Virus)-[:CAUSES]->(s:Symptom) WHERE s.name CONTAINS 'Respiratory' OR s.name CONTAINS 'Gastrointestinal' RETURN DISTINCT v.name"},
    {"question": "Which articles mention both respiratory and cardiovascular symptoms?", "query": "MATCH (a:Article)-[:MENTIONS_SYMPTOM]->(s:Symptom) WHERE s.name CONTAINS 'Respiratory' OR s.name CONTAINS 'Cardiovascular' RETURN a.url"},
    {"question": "Which organizations are involved in publishing articles about viruses affecting Asia?", "query": "MATCH (a:Article)-[:MENTIONS_LOCATION]->(l:Location) WHERE l.country IN ['China', 'India', 'Japan']-[:PUBLISHED_BY]->(o:Organization) RETURN DISTINCT o.name, a.url"},
    {"question": "Which articles mention viruses transmitted by both animals and direct contact?", "query": "MATCH (a:Article)-[:ABOUT_VIRUS]->(v:Virus)-[:SPREADS_BY]->(t:Transmission) WHERE t.method IN ['Animal', 'Direct Contact'] RETURN DISTINCT a.url"},
    {"question": "Which symptoms are most commonly mentioned in articles about viruses affecting the United States?", "query": "MATCH (a:Article)-[:MENTIONS_SYMPTOM]->(s:Symptom)-[:MENTIONS_LOCATION]->(l:Location {country: 'United States'}) RETURN s.name, COUNT(a) AS mention_count ORDER BY mention_count DESC LIMIT 1"},

    # Further combinations
    {"question": "Which articles mention viruses affecting both North America and Asia?", "query": "MATCH (a:Article)-[:MENTIONS_LOCATION]->(l:Location) WHERE l.country IN ['United States', 'China'] RETURN DISTINCT a.url"},
    {"question": "Which transmission methods are mentioned in articles about viruses causing respiratory symptoms?", "query": "MATCH (a:Article)-[:ABOUT_VIRUS]->(v:Virus)-[:SPREADS_BY]->(t:Transmission)-[:MENTIONS_SYMPTOM]->(s:Symptom {name: 'Respiratory'}) RETURN t.method"},
    {"question": "Which organizations are involved in viruses affecting multiple continents?", "query": "MATCH (o:Organization)-[:INVOLVED_IN]->(v:Virus)-[:AFFECTS]->(l:Location) WITH o, COUNT(DISTINCT l.country) AS continent_count WHERE continent_count > 1 RETURN o.name, continent_count"},
    {"question": "What symptoms are mentioned in articles published by WHO?", "query": "MATCH (a:Article)-[:PUBLISHED_BY]->(o:Organization {name: 'WHO'})-[:MENTIONS_SYMPTOM]->(s:Symptom) RETURN s.name, COUNT(a) AS mention_count ORDER BY mention_count DESC"},
    {"question": "Which transmission methods are associated with articles mentioning cardiovascular symptoms?", "query": "MATCH (a:Article)-[:MENTIONS_SYMPTOM]->(s:Symptom {name: 'Cardiovascular'})-[:ABOUT_VIRUS]->(v:Virus)-[:SPREADS_BY]->(t:Transmission) RETURN t.method"},

    # Wrapping up with some general queries
    {"question": "Which viruses are mentioned in articles labeled as true?", "query": "MATCH (a:Article {label: 'True'})-[:ABOUT_VIRUS]->(v:Virus) RETURN DISTINCT v.name"},
    {"question": "Which viruses are mentioned in articles labeled as fake?", "query": "MATCH (a:Article {label: 'Fake'})-[:ABOUT_VIRUS]->(v:Virus) RETURN DISTINCT v.name"},
    {"question": "What are the most common symptoms mentioned in articles about viruses affecting Europe?", "query": "MATCH (a:Article)-[:MENTIONS_SYMPTOM]->(s:Symptom)-[:MENTIONS_LOCATION]->(l:Location) WHERE l.country IN ['United Kingdom', 'France', 'Germany'] RETURN s.name, COUNT(a) AS mention_count ORDER BY mention_count DESC LIMIT 5"},
    {"question": "What viruses are mentioned in articles labeled as fact?", "query": "MATCH (a:Article {label: 'True'})-[:ABOUT_VIRUS]->(v:Virus) RETURN v.name"},
    {"question": "Which viruses affect both Mexico and the United Kingdom?", "query": "MATCH (v:Virus)-[:AFFECTS]->(l:Location) WHERE l.country IN ['Mexico', 'United Kingdom'] RETURN DISTINCT v.name"},

    # Finishing up with 100th question
    {"question": "Which organizations are involved in viruses that affect multiple countries?", "query": "MATCH (o:Organization)-[:INVOLVED_IN]->(v:Virus)-[:AFFECTS]->(l:Location) WITH o, COUNT(DISTINCT l.country) AS country_count WHERE country_count > 1 RETURN o.name, country_count"}
]

questions_and_queries += [
    # More virus-specific questions
    {"question": "Which viruses affect both Africa and Asia?", "query": "MATCH (v:Virus)-[:AFFECTS]->(l:Location) WHERE l.country IN ['Nigeria', 'South Africa', 'China', 'India'] RETURN DISTINCT v.name"},
    {"question": "Which viruses cause both neurological and gastrointestinal symptoms?", "query": "MATCH (v:Virus)-[:CAUSES]->(s:Symptom) WHERE s.name CONTAINS 'Neurological' OR s.name CONTAINS 'Gastrointestinal' RETURN DISTINCT v.name"},
    {"question": "Which articles mention viruses that affect both Brazil and the United States?", "query": "MATCH (a:Article)-[:MENTIONS_LOCATION]->(l:Location) WHERE l.country IN ['Brazil', 'United States'] RETURN DISTINCT a.url"},

    # More symptom-specific questions
    {"question": "Which viruses are responsible for fever and rash symptoms in Mexico?", "query": "MATCH (v:Virus)-[:CAUSES]->(s:Symptom), (v)-[:AFFECTS]->(l:Location {name: 'Mexico'}) WHERE s.name IN ['Fever', 'Rash'] RETURN DISTINCT v.name"},
    {"question": "Which viruses cause symptoms related to skin issues?", "query": "MATCH (v:Virus)-[:CAUSES]->(s:Symptom) WHERE s.name CONTAINS 'Skin' RETURN DISTINCT v.name"},

    # Transmission and symptom combination
    {"question": "Which viruses are spread by airborne transmission and cause respiratory symptoms?", "query": "MATCH (v:Virus)-[:SPREADS_BY]->(t:Transmission {method: 'Airborne'})-[:CAUSES]->(s:Symptom {name: 'Respiratory'}) RETURN DISTINCT v.name"},

    # Final organization and location query
    {"question": "Which organizations are involved in viruses affecting both Africa and South America?", "query": "MATCH (o:Organization)-[:INVOLVED_IN]->(v:Virus)-[:AFFECTS]->(l:Location) WHERE l.country IN ['Nigeria', 'Brazil'] RETURN DISTINCT o.name"}
]


data = pd.DataFrame(questions_and_queries)

In [ ]:
data.to_csv("data_for_evaluate_models.csv", index=False)

In [ ]:
from langchain.graphs import Neo4jGraph

url = NEO4J_URL
username =NEO4J_USERNAME
password = NEO4J_PASSWORD

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password

)

In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_experimental.graph_transformers import LLMGraphTransformer

##Hyperparameters

TEMPERATURE_MODEL=0.5

llm_llama37b = ChatGroq(
    model="llama3-70b-8192",
    temperature=TEMPERATURE_MODEL,
    groq_api_key="",
)

llm_mixtral7b = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=TEMPERATURE_MODEL,
    groq_api_key="",
)

llm_gemma7b = ChatGroq(
    model="gemma-7b-it",
    temperature=TEMPERATURE_MODEL,
    groq_api_key="",
)

llm_llama8b = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=TEMPERATURE_MODEL,
    groq_api_key="",
)


In [ ]:
def generate_code(question):
  cypher_chain = GraphCypherQAChain.from_llm(
      graph=graph,
      cypher_llm=llm_llama8b,
      qa_llm=llm_llama8b,
      validate_cypher=True,
      verbose=True,
      allow_dangerous_requests=True,
      return_intermediate_steps=True
  )
  try:
    response =  cypher_chain.invoke({"query": question})
    query_final = response['intermediate_steps'][0]["query"]
    ##result
    result_db = len(response['intermediate_steps'][1]["context"])
    if result_db>0:
      retrieve_data = 1
    else:
      retrieve_data = 0
    return query_final, retrieve_data
  except:
    return "The query does not generate!" , 0



In [ ]:
generate_code(data.iloc[0]["question"])

Generated Cypher:
MATCH (v:Virus) RETURN v
Full Context:
[{'v': {'name': 'Mpox, COVID-19'}}, {'v': {'name': 'Monkeypox'}}, {'v': {'name': 'Mpox'}}, {'v': {'name': 'Mpox, Monkeypox'}}, {'v': {'name': 'Mpox, Monkeypox, COVID-19'}}, {'v': {'name': 'Monkeypox, COVID-19'}}, {'v': {'name': 'COVID-19'}}, {'v': {'name': 'Monkeypox, COVID-19, SARS-CoV-2'}}]

> Finished chain.


('MATCH (v:Virus) RETURN v', 1)

In [ ]:
# Assuming generate_code and data_filtered are already defined
from tqdm import tqdm
import pandas as pd
import evaluate
def compare_results_first(x1,x2):
  if x1.lower()==x2.lower():
    return 1
  else:
    return 0

# Initialize tqdm to track progress
tqdm.pandas()

data_copy = data.copy()
model = "llama8b"

i=0
band = [20,40,60,80]

# Iterate over each row with progress tracking using iterrows()
for index, row in tqdm(data_copy.iterrows(), total=len(data_copy)):
    # Apply the generate_code function to the 'query' column
    query_result, retrieve_data = generate_code(row["question"])
    # Assign the results to the corresponding columns
    data_copy.at[index, f"answer_{model}"] = query_result
    data_copy.at[index, f"retrieve_data_{model}"] = retrieve_data
    if i in band:
      data_copy.to_csv(f"partial_with_answers_{model}_tmp_{TEMPERATURE_MODEL}.csv", index=False)
      i+=1
    else:
      i+=1



  0%|          | 0/100 [00:00<?, ?it/s]WARNING:langchain_core.callbacks.manager:Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Generated Cypher:
MATCH (v:Virus) RETURN v.name AS VirusName
Full Context:
[{'VirusName': 'Mpox, COVID-19'}, {'VirusName': 'Monkeypox'}, {'VirusName': 'Mpox'}, {'VirusName': 'Mpox, Monkeypox'}, {'VirusName': 'Mpox, Monkeypox, COVID-19'}, {'VirusName': 'Monkeypox, COVID-19'}, {'VirusName': 'COVID-19'}, {'VirusName': 'Monkeypox, COVID-19, SARS-CoV-2'}]


  1%|          | 1/100 [00:00<00:46,  2.12it/s]WARNING:langchain_core.callbacks.manager:Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Generated Cypher:
MATCH (v:Virus {name: 'Mpox'})-[:CAUSES]->(s:Symptom) RETURN s.name


  2%|▏         | 2/100 [00:00<00:40,  2.42it/s]WARNING:langchain_core.callbacks.manager:Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


...


> Finished chain.
Generated Cypher:
MATCH (v:Virus)-[:SPREADS_BY]->(t:Transmission {method: 'airborne'}), (v)-[:CAUSES]->(s:Symptom {name: 'respiratory'}) RETURN v.name
Full Context:
[]


 99%|█████████▉| 99/100 [07:20<00:04,  4.40s/it]WARNING:langchain_core.callbacks.manager:Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Generated Cypher:
MATCH (l1:Location {name: "Africa"})<-[:AFFECTS]-(v:Virus)-[:CAUSES]->(s:Symptom)<-[:CAUSES]-(v2:Virus)-[:AFFECTS]->(l2:Location {name: "South America"}), (o:Organization)-[:INVOLVED_IN]->(v2) RETURN o.name
Full Context:
[]


100%|██████████| 100/100 [07:24<00:00,  4.45s/it]


> Finished chain.


In [ ]:

def compare_cypher_queries(true_query: str, generated_query: str) -> float:

    # Load the BLEU metric from the evaluate library
    bleu_metric = evaluate.load("bleu")

    # Prepare queries in list format for BLEU evaluation
    references = [true_query]  # Reference should be a list of strings (not tokenized)
    predictions = [generated_query]  # Prediction should also be a string

    try:
      # Calculate the BLEU score
      result = bleu_metric.compute(predictions=predictions, references=references)
    except:
      return 0.0
    # Return the BLEU score
    return result['bleu']
def compare_results_first(x1,x2):
  if x1.lower()==x2.lower():
    return 1
  else:
    return 0

In [ ]:
data_copy[f"answer_{model}"] = data_copy[f"answer_{model}"].str.replace("cypher","")
data_copy[f"answer_{model}"] = data_copy[f"answer_{model}"].str.replace("\n","")

data_copy[f'BLEU_score_{model}'] = data_copy.apply(lambda row: compare_cypher_queries(str(row['query']), str(row[f"answer_{model}"])), axis=1)
data_copy[f'SQ_score_{model}'] = data_copy.apply(lambda row: compare_results_first(str(row['query']), str(row[f"answer_{model}"])), axis=1)

data_copy.to_csv(f"data_with_answers_with_comparisson_{model}_tmp_{TEMPERATURE_MODEL}.csv", index=False)


In [ ]:
data_copy

,question,query,answer_gemma7b,retrieve_data_gemma7b,BLEU_score_gemma7b,SQ_score_gemma7b
0,What are all the viruses mentioned in the data...,MATCH (v:Virus) RETURN v.name,MATCH (v:Virus) RETURN v.name,1.0,1.000000,1
1,Which symptoms are caused by the virus Mpox?,MATCH (v:Virus {name: 'Mpox'})-[:CAUSES]->(s:S...,MATCH (virus:Virus {name: 'Mpox'})-[:CAUSES]->...,1.0,0.644351,0
2,What locations are affected by Mpox?,MATCH (v:Virus {name: 'Mpox'})-[:AFFECTS]->(l:...,MATCH (v:Virus {name: 'Mpox'})-[:AFFECTS]->(l:...,1.0,0.862303,0
3,What transmission methods are related to Mpox?,MATCH (v:Virus {name: 'Mpox'})-[:SPREADS_BY]->...,MATCH (v:Virus {name: 'Mpox'})-[:SPREADS_BY]->...,1.0,1.000000,0
4,Which organizations are involved in Mpox?,MATCH (v:Virus {name: 'Mpox'})<-[:INVOLVED_IN]...,MATCH (v:Virus {name: 'Mpox'})<-[:INVOLVED_IN]...,1.0,1.000000,0
...,...,...,...,...,...,...
95,Which articles mention viruses that affect bot...,MATCH (a:Article)-[:MENTIONS_LOCATION]->(l:Loc...,,0.0,0.000000,0
96,Which viruses are responsible for fever and ra...,"MATCH (v:Virus)-[:CAUSES]->(s:Symptom), (v)-[:...",MATCH (virus:Virus)-[:CAUSES]->(symptom:Sympto...,0.0,0.320604,0
97,Which viruses cause symptoms related to skin i...,MATCH (v:Virus)-[:CAUSES]->(s:Symptom) WHERE s...,MATCH (v:Virus)-[:CAUSES]->(s:Symptom) WHERE s...,0.0,0.660109,0
98,Which viruses are spread by airborne transmiss...,MATCH (v:Virus)-[:SPREADS_BY]->(t:Transmission...,MATCH (v:Virus)-[:SPREADS_BY]->(t:Transmission...,0.0,0.598491,0
